## Load dataset


In [2]:
import torch, json, os, logging
import numpy as np
from hmc.dataset import initialize_dataset
from torch.utils.data import Dataset
import networkx as nx
import logging

In [3]:
from hmc.utils import __load_json__

In [4]:
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

In [5]:
import pandas as pd

In [6]:
dataset_path = '/home/bruno/storage/data/datasets'

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
# Criar um logger
logger = logging.getLogger(__name__)

In [9]:
ds_train, ds_val, ds_test = initialize_dataset('seq_GO', dataset_path, output_path='/home/bruno/storage/data/outputs', is_local=True, is_global=True)

  0%|          | 0/5839 [00:00<?, ?it/s]

2025-02-23 17:50:54,665 - INFO - Terminou: 2


  0%|          | 0/5839 [00:00<?, ?it/s]

2025-02-23 17:51:03,264 - INFO - Terminou: 2


  0%|          | 0/5839 [00:00<?, ?it/s]

2025-02-23 17:51:07,615 - INFO - Terminou: 2


In [ ]:
ds_test.nodes_idx.get("")

In [28]:
ds_test.df

,features,categories
0,"[4.8, 0.7, 3.1, 3.6, 3.9, 5.5, 1.4, 6.0, 8.0, ...",GO0004519@GO0005739@GO0008150
1,"[5.2, 0.5, 3.6, 2.3, 6.5, 6.5, 1.4, 7.7, 7.9, ...",GO0004519@GO0005739@GO0006314@GO0008380
2,"[4.3, 1.1, 2.5, 2.2, 5.7, 6.3, 2.4, 9.2, 6.0, ...",GO0004519@GO0005739@GO0006316
3,"[5.7, 1.0, 2.1, 1.3, 9.4, 6.5, 3.4, 10.4, 1.8,...",GO0008121@GO0005750@GO0005739@GO0006122@GO0009060
4,"[2.9, 1.2, 3.3, 2.3, 5.4, 5.0, 1.9, 11.4, 8.9,...",GO0003723@GO0005739@GO0000372
...,...,...
1327,"[3.2, 0.0, 7.6, 8.3, 2.5, 4.5, 1.3, 7.6, 5.1, ...",GO0016455@GO0000119@GO0006366
1328,"[8.1, 1.4, 6.0, 4.6, 5.1, 4.9, 2.3, 5.7, 4.0, ...",GO0005096@GO0030127@GO0005737@GO0030433@GO0006...
1329,"[3.5, 1.2, 2.3, 8.1, 5.8, 7.0, 2.3, 4.7, 4.7, ...",GO0031202@GO0005682@GO0005685@GO0046540@GO0000398
1330,"[2.6, 0.5, 5.8, 5.4, 3.3, 4.9, 3.0, 5.3, 5.8, ...",GO0005515@GO0019898@GO0006914@GO0006623


In [37]:
ds_test.Y[3][762]

np.float64(0.0)

In [36]:
ds_test.nodes_idx.get("GO0004519")

762

In [358]:
teste = ['12', '85', '404']

In [359]:
teste[:0]

[]

In [12]:
from collections import defaultdict

def build_complete_paths(pairs):
    """
    Constrói caminhos completos a partir de pares pai-filho.

    :param pairs: Lista de strings representando relações pai-filho (exemplo: "A.B").
    :return: Lista de strings representando os caminhos completos.
    """
    child_to_parent = {}
    roots = set()
    all_nodes = set()

    # Construir dicionário de relação filho -> pai
    for pair in pairs:
        parent, child = pair.split('.')
        child_to_parent[child] = parent
        all_nodes.update([parent, child])

    # Identificar raízes (nós que não são filhos de ninguém)
    roots = all_nodes - set(child_to_parent.keys())

    # Reconstruir os caminhos completos
    complete_paths = []
    for root in roots:
        stack = [(root, root)]  # (nó atual, caminho acumulado)

        while stack:
            node, path = stack.pop()
            children = [c for c, p in child_to_parent.items() if p == node]

            if children:
                for child in children:
                    stack.append((child, f"{path}.{child}"))
            else:
                complete_paths.append(path)  # Nó folha -> salvar caminho completo

    return complete_paths


In [16]:

labels = [
     "GO0004003.GO0043141", "GO0003678.GO0043138", "GO0043138.GO0043140", "GO0003678.GO0043139", "GO0043139.GO0043141", "GO0004386.GO0003724"
]

In [ ]:
def load_structure(self, labels):
    # Load labels JSON
    g = nx.DiGraph()
    for cat in tqdm(labels):
        terms = cat.split('.')
    
        g.add_edge(terms[1], terms[0])
        
    nodes = sorted(self.g.nodes(), key=lambda x: (len(x.split('.')), x))
    nodes_idx = dict(zip(self.nodes, range(len(self.nodes))))
    g_t = self.g.reverse()
    
    ### Save networkx graph
    # Para salvar em formato GraphML
    #nx.write_graphml(self.g, self.graph_path)
    
    A = nx.to_numpy_array(self.g, nodelist=self.nodes)
    

In [26]:
nx.shortest_path_length(ds_train.g, "GO0043139", 'root')

5

In [31]:
nx.ancestors(ds_train.g_t, "root")

{'GO0015822',
 'GO0015037',
 'GO0016811',
 'GO0000087',
 'GO0031124',
 'GO0003913',
 'GO0030004',
 'GO0009895',
 'GO0042575',
 'GO0031578',
 'GO0005976',
 'GO0042434',
 'GO0042726',
 'GO0006383',
 'GO0009890',
 'GO0005342',
 'GO0004037',
 'GO0003756',
 'GO0000010',
 'GO0008219',
 'GO0009353',
 'GO0016273',
 'GO0016255',
 'GO0032156',
 'GO0045912',
 'GO0006334',
 'GO0015171',
 'GO0016841',
 'GO0019650',
 'GO0000808',
 'GO0008375',
 'GO0046156',
 'GO0015693',
 'GO0015824',
 'GO0032147',
 'GO0003885',
 'GO0005834',
 'GO0030427',
 'GO0007000',
 'GO0017125',
 'GO0004024',
 'GO0051129',
 'GO0016744',
 'GO0000388',
 'GO0005802',
 'GO0030543',
 'GO0003904',
 'GO0051300',
 'GO0003905',
 'GO0000214',
 'GO0000429',
 'GO0043141',
 'GO0006843',
 'GO0006425',
 'GO0015855',
 'GO0030815',
 'GO0015802',
 'GO0007126',
 'GO0004477',
 'GO0000307',
 'GO0015744',
 'GO0006098',
 'GO0016782',
 'GO0000150',
 'GO0046526',
 'GO0031569',
 'GO0004839',
 'GO0018063',
 'GO0044248',
 'GO0006108',
 'GO0051293',
 'GO00

In [17]:
structured_paths = build_complete_paths(labels)
print(structured_paths)

['GO0003678.GO0043139.GO0043141', 'GO0003678.GO0043138.GO0043140', 'GO0004003', 'GO0004386.GO0003724']


In [16]:
ds_test.df.categories.values

array(['GO0004519@GO0005739@GO0008150',
       'GO0004519@GO0005739@GO0006314@GO0008380',
       'GO0004519@GO0005739@GO0006316', ...,
       'GO0031202@GO0005682@GO0005685@GO0046540@GO0000398',
       'GO0005515@GO0019898@GO0006914@GO0006623',
       'GO0004402@GO0005737@GO0016573'], dtype=object)

In [283]:
df_teste = pd.read_csv('/home/bruno/storage/data/datasets/fma_rock_electronic/test.csv', sep="|")

In [9]:
df_teste

,track_id,file_path,track_genre_top,categories
0,62870,/home/bruno/storage/data/fma/fma_large/062/062...,Rock,12.25.89
1,128705,/home/bruno/storage/data/fma/fma_large/128/128...,Rock,12.88@12.85
2,29500,/home/bruno/storage/data/fma/fma_large/029/029...,Rock,12.25.111@12.85
3,107152,/home/bruno/storage/data/fma/fma_large/107/107...,Electronic,15.184
4,4172,/home/bruno/storage/data/fma/fma_large/004/004...,Rock,12.25@12.27
...,...,...,...,...
104,60832,/home/bruno/storage/data/fma/fma_large/060/060...,Electronic,15
105,1746,/home/bruno/storage/data/fma/fma_large/001/001...,Rock,12
106,55906,/home/bruno/storage/data/fma/fma_large/055/055...,Rock,12.27@12.70
107,43157,/home/bruno/storage/data/fma/fma_large/043/043...,Electronic,15.468


In [11]:
fma_path = os.path.join(dataset_path, 'fma_rock_electronic')

In [12]:
labels_json = os.path.join(fma_path, 'labels.json')

In [13]:
labels_json.split('/')[-1]

'labels.json'

In [294]:
t = "12.20.30.40"

In [300]:
nodes = t.split(".")
for idx in range(1, len(nodes)+1):
    local_label = nodes[:idx]
    if len(local_label) > 1:
        local_label = ".".join(local_label)
    else:
        local_label = local_label[0]
    
    print(local_label)
    print(ds_train.local_nodes_idx[idx-1])

12
{'15.183': 0, '12.25': 1, '12.98': 2, '15.495': 3, '15.468': 4, '15.695': 5, '15.337': 6, '12.45': 7, '12.31': 8, '15.297': 9, '12.66': 10, '12.359': 11, '12.314': 12, '15.181': 13, '15.42': 14, '15.182': 15, '12.85': 16, '15.296': 17, '12.27': 18, '12.88': 19, '15.185': 20, '15.184': 21, '15.286': 22, '12.58': 23, '12.26': 24, '15.236': 25, '12.70': 26}
12.20
{'15': 0, '12': 1}
12.20.30
{'15.468.491': 0, '12.25.71': 1, '12.31.101': 2, '12.31.167': 3, '12.25.89': 4, '12.45.53': 5, '15.181.401': 6, '15.182.400': 7, '12.85.404': 8, '12.25.111': 9, '15.297.240': 10, '12.25.64': 11, '12.26.113': 12, '12.25.109': 13, '12.31.439': 14}
12.20.30.40
{'12.25.109.361': 0}


In [296]:
nodes = t.split(".")
for idx in range(1, len(nodes)+1):
    local_label = nodes[:idx]
    if len(local_label) > 1:
        local_label = ".".join(local_label)
    else:
        local_label = local_label[0]
    print(self.local_nodes_idx[idx].get(local_label))

12
12.20
12.20.30
12.20.30.40


In [16]:
elements['nodes']

['12',
 '15',
 '12.25',
 '12.26',
 '12.27',
 '12.31',
 '12.314',
 '12.359',
 '12.45',
 '12.58',
 '12.66',
 '12.70',
 '12.85',
 '12.88',
 '12.98',
 '15.181',
 '15.182',
 '15.183',
 '15.184',
 '15.185',
 '15.236',
 '15.286',
 '15.296',
 '15.297',
 '15.337',
 '15.42',
 '15.468',
 '15.495',
 '15.695',
 '12.25.109',
 '12.25.111',
 '12.25.64',
 '12.25.71',
 '12.25.89',
 '12.26.113',
 '12.31.101',
 '12.31.167',
 '12.31.439',
 '12.45.53',
 '12.85.404',
 '15.181.401',
 '15.182.400',
 '15.297.240',
 '15.468.491',
 '12.25.109.361']

In [17]:
elements['A'][4]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [18]:
elements['graph']

In [19]:
for a in nx.ancestors(elements['g_t'], '15.183'):
    print(a)

15


In [43]:
df_teste.categories

0      12.25.89
1      12.25.89
2      12.25.89
3      12.25.89
4      12.25.89
         ...   
104    12.25.89
105    12.25.89
106    12.25.89
107    12.25.89
108    12.25.89
Name: categories, Length: 109, dtype: object

In [44]:
Y = []

for df_teste.categories in df_teste.categories.values:
    y_ = np.zeros(len(elements['nodes']))
    for t in labels.split('@'):
        y_[[elements['nodes_idx'].get(a) for a in nx.ancestors(elements['g_t'], t)]] = 1
        y_[elements['nodes_idx'][t]] = 1
    Y.append(y_)
Y = np.stack(Y)

In [45]:
Y[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [65]:
features_path = os.path.join(dataset_path, 'fma', 'fma_large')

In [16]:
import os
import pandas as pd
import tensorflow as tf

In [33]:
def parse_tfr_element(element):
    # use the same structure as above; it's kinda an outline of the structure we now want to create
    data = {
        'emb': tf.io.FixedLenFeature([], tf.string),
        'track_id': tf.io.FixedLenFeature([], tf.int64),
    }

    content = tf.io.parse_single_example(element, data)

    track_id = content['track_id']
    emb = content['emb']

    # get our 'feature'-- our image -- and reshape it appropriately
    features = tf.io.parse_tensor(emb, out_type=tf.float32)
    return (features, track_id)


In [35]:
def load_features(dataset_path):
    dataset_path = [os.path.join(dataset_path, path) for path in os.listdir(dataset_path) if path.endswith('.tfrecord')]
    dataset = get_dataset(dataset_path)

    df = pd.DataFrame(
        dataset.as_numpy_iterator(),
        columns=['features', 'track_id']
    )

    df.dropna(inplace=True)

    
    return df

In [36]:
df = load_features(features_path)

2025-02-02 17:05:02.589689: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [37]:
df

,features,track_id
0,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,2
1,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,3
2,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,5
3,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,10
4,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,20
...,...,...
104181,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,138179
104182,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,138180
104183,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,138181
104184,b'\x08\x01\x12\t\x12\x02\x08\x1d\x12\x03\x08\x...,138182


In [12]:
ds_train.df

,features,categories
0,"[0.15, None, -0.22, 0.07, -0.15, -0.15, -0.21,...",GO0003709@GO0000127@GO0005739@GO0006384
1,"[-1.22, -0.27, -0.1, 0.23, -0.14, -0.71, 0.1, ...",GO0003746@GO0005840@GO0005853@GO0006414
2,"[-0.6, 1.01, 0.24, 0.65, -0.05, -0.53, -0.47, ...",GO0016887@GO0051082@GO0005832@GO0005634@GO0000...
3,"[0.25, -0.79, -0.22, -0.54, -0.03, -0.27, 0.17...",GO0003674@GO0016021@GO0042175@GO0006997@GO0007...
4,"[-0.12, -0.54, -0.12, -0.18, 0.0, -0.01, 0.12,...",GO0000703@GO0008534@GO0005634@GO0005739@GO0006...
...,...,...
1620,"[0.28, None, -0.45, -0.27, 0.04, 0.2, -0.18, 0...",GO0003674@GO0005575@GO0008150
1621,"[-0.18, -0.25, -0.03, -0.17, 0.23, -0.03, 0.43...",GO0015239@GO0005887@GO0042493@GO0015893
1622,"[0.12, -0.3, -0.14, 0.19, 0.14, 0.09, 0.25, -0...",GO0003702@GO0005634@GO0005737@GO0046685@GO0045944
1623,"[0.14, -1.18, -0.27, -0.18, -0.09, 0.04, -0.06...",GO0030611@GO0005575@GO0046685


In [10]:
len(ds_train.Y[0])

4126

In [11]:
ds_train.Y[0]

array([1., 1., 1., ..., 0., 1., 0.])

In [7]:
scaler = preprocessing.StandardScaler().fit(np.concatenate((ds_train.X_cont, ds_val.X_cont)))
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean').fit(np.concatenate((ds_train.X_cont, ds_val.X_cont)))

In [8]:
ds_val.X_count, ds_val.Y = scaler.transform(imp_mean.transform(ds_val.X_cont)), torch.tensor(ds_val.Y).to(device)
ds_train.X_count, ds_train.Y = scaler.transform(imp_mean.transform(ds_train.X_cont)), torch.tensor(ds_train.Y).to(device)